In [1]:
import os
import traceback
import logging
import os.path
import networkx as nx
import pandas as pd
import numpy as np
import pylab as pl
import glob
import re
import csv
import shutil

In [13]:
## Sequence data directory (please download the data in put in the dir: Ecoli1936)
pangenome_data = '../../data/Ecoli1936/'

In [3]:
if 'Kp' in pangenome_data:
    data_base = ' --genus Klebsiella --species pneumoniae --cpus 30'
elif 'Ecoli' in pangenome_data:
    data_base = ' --genus Escherichia --species coli --cpus 30'
else:
    data_base = ' --cpus 30'
print("Use the correct database for the species: ", data_base)

Use the correct database for the species:   --genus Escherichia --species coli --cpus 30


### Run Prokka (using command lines or this code)

In [ ]:
output_dir = pangenome_data + 'temp'
prokka_dir = pangenome_data + 'prokka_revision'
os.system('mkdir '+ prokka_dir)
conda_dir = 'source ~/miniconda3/etc/profile.d/conda.sh && conda activate amromics && ' # load python environment where prokka is installed
fffile = pangenome_data + '*.fa'
list_database = glob.glob(fffile)
for i in range(len(list_database)):
    input_data = list_database[i]
    print(input_data)
    file_name = input_data.split("/")[-1][:-4]
    prokka_bin = conda_dir + 'prokka --outdir ' + output_dir +data_base +' --prefix ' + file_name +' '+input_data
    os.system(prokka_bin)
    os.system('cp ' + output_dir + '/'+file_name+'.gff ' + prokka_dir)
    os.system('rm -r '+ output_dir)
    # if i>=1:
    #     break;

### Prepare data for panta -add with sequence type stratification

In [6]:
## Run MLST with the command line:
# mlst --csv --scheme ecoli ../../data/Ecoli1936/*.fa > Ecolimlst.csv
## To obtain the metadata please check the notebook ST_analysis.ipynb

In [11]:
metadata_panta = pd.read_csv("../../data/metadata/metadataEcoliST.csv")
metadata_panta.head(2)

,Isolate,Year,CTZ,CTX,AMP,AMX,AMC,TZP,CXM,CET,GEN,TBM,TMP,CIP,ST
0,11658_4#1,2006.0,S,S,S,NaN,S,S,R,S,S,S,S,S,135
1,11657_5#1,2006.0,S,S,R,NaN,R,S,S,S,S,S,R,R,10


In [12]:
sample_isolate = pd.read_csv('../../data/metadata/sample_isolate.csv')
sample_isolate.head(2)
isolate2sample = {}
for idx in range(len(sample_isolate.index)):
    isolate2sample[sample_isolate.iloc[idx,1]] =  sample_isolate.iloc[idx,0]+'.contig.gff'

In [4]:
isolate2sample['11658_4#1']

'SAMEA2204229.contig.gff'

In [5]:
test1_idx = [73, 10, 405, 38, 393]
train_dir = '../../data/Ecoli1936/prokka_train/train'
test_dir = '../../data/Ecoli1936/prokka_test/test'
if not os.path.isdir(train_dir):
    os.system('mkdir '+ train_dir)
if not os.path.isdir(test_dir):
    os.system('mkdir '+ test_dir)

In [6]:
for i in range(len(metadata_panta.index)):
    filedir = '/data/hoan/amromics/prediction/data/Ecoli1936/prokka/' + isolate2sample[metadata_panta.iloc[i, 0]]
    cmd1 = 'cp '+ filedir + ' ' + train_dir
    cmd2 = 'cp '+ filedir + ' ' + test_dir
    if metadata_panta.iloc[i, -1] in test1_idx:
        os.system(cmd2)
    else:
        os.system(cmd1)

### Then, run panta

In [2]:
# Instruction to run panta here: https://github.com/amromics/panta

In [1]:
# Split the data into train and test directory (by random or stratified by sequence type)

In [1]:
# Run the following command lines from the terminal
panta main -o pantaTrainAdd -g prokka_train/train/*.gff -a protein -s -i 0.65
cp -r pantaTrainAdd pantaTrain
panta add -c pantaTrainAdd -g prokka_test/test/*.gff -a protein -s -i 0.65
cp -r pantaTrainAdd pantaTest